In [14]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.108.0
    Uninstalling openai-1.108.0:
      Successfully uninstalled openai-1.108.0


In [3]:
import openai
from PyPDF2 import PdfReader
import docx
import os  # For file path and existence checks
from google.colab import files  # For file uploads in Colab

# Install required libraries (if not already installed)
!pip install PyPDF2 python-docx openai

# Load API key securely from Colab Secrets
from google.colab import userdata

secret_name = "OPENAI_API_KEY"
try:
    key = userdata.get(secret_name)
    if not key:
        raise ValueError(f"Secret {secret_name} is empty.")
    # Clean the key and set as environment variable
    key = key.strip()
    os.environ["OPENAI_API_KEY"] = key
    openai.api_key = key
    print("✅ OPENAI_API_KEY loaded successfully.")

except userdata.SecretNotFoundError:
    print(f"Error: Secret '{secret_name}' not found in Colab Secrets.")
    print("Open the left-side 'Secrets' pane, add the secret, and grant Notebook access.")
    openai.api_key = None
except ValueError as e:
    print(f"Error loading API key: {e}")
    openai.api_key = None
except Exception as e:
    print(f"Unexpected error while loading API key: {e}")
    openai.api_key = None


# Function to read a TXT file
def read_txt(file_path):
    if not os.path.exists(file_path):
        print(f"Error: File not found at {file_path}")
        return None
    with open(file_path, "r", encoding="utf-8") as f:
        return f.read()


# Function to read a PDF file
def read_pdf(file_path):
    if not os.path.exists(file_path):
        print(f"Error: File not found at {file_path}")
        return None
    try:
        reader = PdfReader(file_path)
        text = ""
        for page in reader.pages:
            try:
                page_text = page.extract_text()
                if page_text:  # Ensure page text is not None
                    text += page_text + "\n"
            except Exception as e:
                print("Error in a PDF page:", e)
                continue
        return text
    except Exception as e:
        print(f"Error reading PDF file: {e}")
        return None


# Function to read a DOCX file
def read_docx(file_path):
    if not os.path.exists(file_path):
        print(f"Error: File not found at {file_path}")
        return None
    try:
        doc = docx.Document(file_path)
        text = ""
        for para in doc.paragraphs:
            text += para.text + "\n"
        return text
    except Exception as e:
        print(f"Error reading DOCX file: {e}")
        return None


# Function to generate questions using OpenAI API
def generate_questions(text, num_questions):
    if not openai.api_key:
        print("Error: OpenAI API key not loaded. Cannot generate questions.")
        return None

    prompt = f"""
    From the following text, create {num_questions} multiple-choice questions:
    ---
    {text}
    ---
    Write each question with 4 options (A, B, C, D) and clearly indicate the correct answer.
    """

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that creates multiple choice questions."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1500,
            temperature=0.7
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error generating questions with OpenAI API: {e}")
        return None


# Main Program
if __name__ == "__main__":
    # Upload file (txt / pdf / docx)
    uploaded = files.upload()

    # Get the name of the first uploaded file
    file_path = list(uploaded.keys())[0]
    print("Uploaded:", file_path)

    content = None
    if file_path.endswith(".txt"):
        content = read_txt(file_path)
    elif file_path.endswith(".pdf"):
        content = read_pdf(file_path)
    elif file_path.endswith(".docx"):
        content = read_docx(file_path)
    else:
        print("Unsupported file format.")

    if content:
        try:
            num_q = int(input("How many questions do you want? (10 / 20 / 30): "))
            if num_q not in [10, 20, 30]:
                print("Invalid number of questions. Please enter 10, 20, or 30.")
            else:
                questions = generate_questions(content, num_q)

                print("\n=== Quiz ===\n")
                print(questions)
        except ValueError:
            print("Invalid input for number of questions. Please enter a number.")


✅ OPENAI_API_KEY loaded successfully.


Saving test file.pdf to test file (4).pdf
Uploaded: test file (4).pdf
How many questions do you want? (10 / 20 / 30): 10
Error generating questions with OpenAI API: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

=== Quiz ===

None


In [10]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 14.8 MB/s eta 0:00:00


In [8]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 11.9 MB/s eta 0:00:00
